In [1]:
import keras
from keras.layers import Layer,Lambda,Dense,Conv2D,BatchNormalization,MaxPooling2D,AveragePooling2D,Activation,Input,concatenate,Flatten
from keras.models import Model
from keras.utils import plot_model
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
import random
import tensorflow as tf
graph = tf.get_default_graph()

Using TensorFlow backend.


**Used AdaBound as an optimizer for compiling the model.
It comes with power of Adam and Stochastic Gradient Descent optimizers for better convergence.**

In [2]:
!pip install keras-adabound
from keras_adabound import AdaBound

  Created wheel for keras-adabound: filename=keras_adabound-0.5.0-cp36-none-any.whl size=6420 sha256=d429de43344b134778dd925bdcf59257af312f209441993bd44d8e4782b09260
  Stored in directory: /root/.cache/pip/wheels/ac/8d/f7/33776ea7ac99b7faf02c27f1be6c129365c0d68799e227021d
Successfully built keras-adabound


**Used Inception v3 model architecture with the following modifications:**

*   The Input image size is 96 x 96.

*   Weight Decay parameter is used with value as 0.125.

*   Batch Normalization and Relu Activation is used for each Convolution Layer.

*   128 Embedding vector is generated using the dense layer after flattening out the output from the previous layers.

*   A Dense Layer with the number of classes is added after that.

*   A SoftMax Activation Layer is added after the above Dense Layer.

In [3]:
alpha=0.125
inputs=Input(shape=(96,96,3))
def inceptionv3():
  def conv2d_with_batchnorm(x,no_of_kernels,kernel_size,strides=(1,1),padding="same"):
	  x=Conv2D(int(alpha*no_of_kernels),kernel_size,strides=strides,padding=padding)(x)
	  x=BatchNormalization()(x)
	  x=Activation("relu")(x)
	  return x
  
  def first_block(x):
	  branch1_1x1=conv2d_with_batchnorm(x,64,(1,1))
	  branch2_1x1=conv2d_with_batchnorm(x,48,(1,1))
	  branch2_5x5=conv2d_with_batchnorm(branch2_1x1,64,((5,5)))
	  branch3_1x1=conv2d_with_batchnorm(x,64,(1,1))
	  branch3_3x3=conv2d_with_batchnorm(branch3_1x1,96,(3,3))
	  branch3_3x3=conv2d_with_batchnorm(branch3_3x3,96,(3,3))
	  avg_pool_block1=AveragePooling2D((3,3),strides=(1,1),padding="same")(x)
	  avg_pool_block1=conv2d_with_batchnorm(avg_pool_block1,32,(1,1))
	  x=concatenate([branch1_1x1,branch2_5x5,branch3_3x3,avg_pool_block1])
	  return x

  def second_block(x,no_of_kernels):
	  branch1_1x1=conv2d_with_batchnorm(x,192,(1,1))
	  branch2_1x1=conv2d_with_batchnorm(x,no_of_kernels,(1,1))
	  branch2_1x7=conv2d_with_batchnorm(branch2_1x1,no_of_kernels,(1,7))
	  branch2_7x1=conv2d_with_batchnorm(branch2_1x7,192,(7,1))
	  branch3_1x1=conv2d_with_batchnorm(x,no_of_kernels,(1,1))
	  branch3_7x1=conv2d_with_batchnorm(branch3_1x1,no_of_kernels,(7,1))
	  branch3_1x7=conv2d_with_batchnorm(branch3_7x1,no_of_kernels,(1,7))
	  branch3_7x1=conv2d_with_batchnorm(branch3_1x7,no_of_kernels,(7,1))
	  branch3_1x7=conv2d_with_batchnorm(branch3_7x1,192,(1,7))
	  avg_pool_block2=AveragePooling2D((3,3),strides=(1,1),padding="same")(x)
	  avg_pool_block2=conv2d_with_batchnorm(avg_pool_block2,192,(1,1))
	  x=concatenate([branch1_1x1,branch2_7x1,branch3_1x7,avg_pool_block2])
	  return x

  def third_block(x):
	  branch1_1x1=conv2d_with_batchnorm(x,320,(1,1))
	  branch2_1x1=conv2d_with_batchnorm(x,384,(1,1))
	  branch2_1x3=conv2d_with_batchnorm(branch2_1x1,384,(1,3))
	  branch2_3x1=conv2d_with_batchnorm(branch2_1x1,384,(3,1))
	  branch2=concatenate([branch2_1x3,branch2_3x1])
	  branch3_1x1=conv2d_with_batchnorm(x,448,(1,1))
	  branch3_3x3=conv2d_with_batchnorm(branch3_1x1,384,(3,3))
	  branch3_1x3=conv2d_with_batchnorm(branch3_3x3,384,(1,3))
	  branch3_3x1=conv2d_with_batchnorm(branch3_3x3,384,(3,1))
	  branch3=concatenate([branch3_1x3,branch3_3x1])
	  avg_pool_block3=AveragePooling2D((3,3),strides=(1,1),padding="same")(x)
	  avg_pool_block3=conv2d_with_batchnorm(avg_pool_block3,192,(1,1))
	  x=concatenate([branch1_1x1,branch2,branch3,avg_pool_block3])
	  return x
  
  x=conv2d_with_batchnorm(inputs,32,(3,3),strides=(2,2),padding="valid")
  x=conv2d_with_batchnorm(x,32,(3,3),padding="valid")
  x=conv2d_with_batchnorm(x,64,(3,3))
  x=MaxPooling2D((3,3),strides=(2,2))(x)
  x=conv2d_with_batchnorm(x,80,(1,1),padding="valid")
  x=conv2d_with_batchnorm(x,192,(3,3),padding="valid")
  x=MaxPooling2D((3,3),strides=(2,2))(x)
  x=first_block(x)
  x=first_block(x)
  branch1_3x3=conv2d_with_batchnorm(x,384,(3,3),strides=(2,2),padding="valid")
  branch2_1x1=conv2d_with_batchnorm(x,64,(1,1))
  branch2_3x3=conv2d_with_batchnorm(branch2_1x1,96,(3,3))
  branch2_3x3=conv2d_with_batchnorm(branch2_3x3,96,(3,3),strides=(2,2),padding="valid")
  max_pool=MaxPooling2D((3,3),strides=(2,2))(x)
  x=concatenate([branch1_3x3,branch2_3x3,max_pool])
  x=second_block(x,128)
  x=second_block(x,160)
  x=second_block(x,160)
  x=second_block(x,192)
  branch1_1x1=conv2d_with_batchnorm(x,192,(1,1))
  branch1_3x3=conv2d_with_batchnorm(branch1_1x1,320,(3,3),strides=(2,2),padding="valid")
  branch2_1x1=conv2d_with_batchnorm(x,192,(1,1))
  branch2_1x7=conv2d_with_batchnorm(branch2_1x1,192,(1,7))
  branch2_7x1=conv2d_with_batchnorm(branch2_1x7,192,(7,1))
  branch2_3x3=conv2d_with_batchnorm(branch2_7x1,192,(3,3),strides=(2,2),padding="valid")
  max_pool=MaxPooling2D((3,3),strides=(2,2))(x)
  x=concatenate([branch1_3x3,branch2_3x3,max_pool])
  x=third_block(x)
  x=third_block(x)
  flat=Flatten()(x)
  dense=Dense(128)(flat)
  dense=Activation("relu")(dense)
  #norm = Lambda(lambda  x: keras.backend.l2_normalize(x, axis=1))(dense)
  outputs=Dense(83)(dense)
  outputs=Activation("softmax")(outputs)
  model=Model(inputs,outputs)
  return model

**Mounting the hosted runtime to colab**

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


**Unzipping Training File from the hosted runtime**

In [0]:
!unzip -q "/content/gdrive/My Drive/Train.zip"

In [0]:
import os
import numpy
import shutil
import cv2
from sklearn.metrics.pairwise import pairwise_distances
import itertools
import gc

**Splitting the training data into train+test set with train ratio 0.9.**

In [7]:
dir_path="/content/content/Train/"
train_path="training/"
test_path="testing/"
count=0
if not os.path.exists(train_path):
  os.mkdir(train_path)
if not os.path.exists(test_path):
  os.mkdir(test_path)
folders=os.listdir(dir_path)
print(folders)
numpy.random.shuffle(folders)
train_rate=0.9
for f in folders:
  if(len(os.listdir(os.path.join(dir_path,f)))>0):
    g=os.listdir(os.path.join(dir_path,f))
    if(len(g)>1):
      train_folder=g[:int(train_rate*len(g))]
      for train in train_folder:
        count=count+1
        print(train)
        if not os.path.exists(train_path+"/"+f):
          os.mkdir(train_path+"/"+f)
        shutil.copy2(os.path.join(dir_path,f,train),(os.path.join(train_path,f)))
      #print(train_folder)
      test_folder=g[int(train_rate*len(g)):]
      for test in test_folder:
        if not os.path.exists(test_path+"/"+f):
          os.mkdir(test_path+"/"+f)
        shutil.copy2(os.path.join(dir_path,f,test),(os.path.join(test_path,f)))
      #print(test_folder)
      #train_target=os.path.join(train_path,f)
      #test_target=os.path.join(test_path,f)

['Venkatesh', 'SureshGopi', 'Jamuna', 'Simran', 'Vishnuvardhan', 'KarunaBenerjee', 'SalmanKhan', 'NedumudiVenu', 'MadhuriDixit', 'Shashikumar', 'SharukhKhan', 'SVR', 'M.S.Narayana', 'Ambresh', 'PrakashRaj', 'Bhanupriya', 'Ali', 'Siddique', 'RakheeGulzar', 'Mammukoya', 'Trisha', 'Ramaprabha', 'Rimisen', 'FaridaJalal', 'Loknath', 'Cochinhaneefa', 'RameshArvind', 'Shobana', 'SoumithraChatterjee', 'AnupamKher', 'Dileep', 'ANR', 'KatrinaKaif', 'KotaSrinivasarao', 'HrithikRoshan', 'Lakshmidevi', 'KareenaKapoor', 'Madhu', 'AkshayKumar', 'Kajol', 'PareshRaval', 'RishiKapoor', 'Mukesh', 'AmrishPuri', 'Brahmanandam', 'Mallikarjunrao', 'Avinash', 'Annapoorna', 'RajeshKhanna', 'Madhavan', 'TanikellaBharani', 'Tenniskrishna', 'Innocent', 'Suryakantham', 'Leelavathi', 'Shivaram', 'Srihari', 'ShilpaShetty', 'K.Viswanath', 'JosePrakash', 'Jayram', 'Balakrishna', 'Prema', 'AmitabhBachchan', 'Umashri', 'NTR', 'RamanaReddy', 'Savithri', 'SharmilaTagore', 'SailendraMukherjee', 'PremaNazir', 'Nagarjuna', '

**Printing number of images in Training Set.**

In [8]:
print(count)

5764


**Archiving the contents of training and testing sets generated from the above.**

In [9]:
!zip -r /content/training.zip /content/training
!zip -r /content/testing.zip /content/testing

  adding: content/training/ (stored 0%)
  adding: content/training/Venkatesh/ (stored 0%)
  adding: content/training/Venkatesh/Venkatesh_71.jpg (deflated 4%)
  adding: content/training/Venkatesh/Venkatesh_35.jpg (deflated 4%)
  adding: content/training/Venkatesh/Venkatesh_29.jpg (deflated 4%)
  adding: content/training/Venkatesh/Venkatesh_59.jpg (deflated 5%)
  adding: content/training/Venkatesh/Venkatesh_82.jpg (deflated 4%)
  adding: content/training/Venkatesh/Venkatesh_113.jpg (deflated 4%)
  adding: content/training/Venkatesh/Venkatesh_9.jpg (deflated 4%)
  adding: content/training/Venkatesh/Venkatesh_90.jpg (deflated 5%)
  adding: content/training/Venkatesh/Venkatesh_45.jpg (deflated 4%)
  adding: content/training/Venkatesh/Venkatesh_48.jpg (deflated 5%)
  adding: content/training/Venkatesh/Venkatesh_103.jpg (deflated 5%)
  adding: content/training/Venkatesh/Venkatesh_110.jpg (deflated 5%)
  adding: content/training/Venkatesh/Venkatesh_32.jpg (deflated 5%)
  adding: content/traini

**Getting the images and the corresponding class labels into a list.**

In [0]:
def get_face_info(dir_path):
  labels=[]
  face=[]
  for label_dir in os.listdir(dir_path):
    full_label_dir=os.path.join(dir_path,label_dir)
    for faces in os.listdir(full_label_dir):
      labels.append(label_dir)
      face.append(cv2.imread(os.path.join(full_label_dir,faces))/255.)
  return face,labels

**Initializing the training and test data with corresponding return values from the above function.**

In [0]:
train_face,train_labels=get_face_info('/content/training/')
test_face,test_labels=get_face_info('/content/testing/')

**Converting list to numpy array for further processing.**

In [0]:
train_face=numpy.array(train_face)
train_face=train_face.reshape(len(train_face),96,96,3)
test_face=numpy.array(test_face)
test_face=test_face.reshape(len(test_face),96,96,3)
train_labels=numpy.array(train_labels)
train_labels=train_labels.reshape(len(train_labels),1)
test_labels=numpy.array(test_labels)
test_labels=test_labels.reshape(len(test_labels),1)
numpy.save('x_train',train_face)
numpy.save('x_test',test_face)
numpy.save('y_train',train_labels)
numpy.save('y_test',test_labels)

**Getting the number of class labels from the corresponding training set.**

In [0]:
nb_classes=numpy.unique(train_labels,axis=0).size
#x_train, x_val, y_train, y_val = train_test_split(train_face, train_labels, test_size=0.1, random_state=random.randint(0, 100))

**Conversion of class labels**



*   Converting labels of string type to integer type by using Label Encoder Pre-processing from sklearn.preprocessing

*   Converting the above labels into categorical encoding using One-Hot Encoding.





In [0]:
from sklearn.preprocessing import LabelEncoder
label_encoder1 = LabelEncoder()
label_encoder2 = LabelEncoder()
label_encoder3 = LabelEncoder()
y_train = label_encoder1.fit_transform(train_labels.ravel())
#y_val = label_encoder2.fit_transform(y_val.ravel())
test_labels = label_encoder3.fit_transform(test_labels.ravel())
y_train=y_train.reshape(len(train_face),1)
#y_val=y_val.reshape(len(x_val),1)
test_labels=test_labels.reshape(len(test_face),1)
y_train = np_utils.to_categorical(y_train, nb_classes)
#y_val = np_utils.to_categorical(y_val, nb_classes)
test_labels = np_utils.to_categorical(test_labels, nb_classes)

**Using Categorical Cross-Entropy as a loss for compiling the model and training the model with batch size as 32.**

In [15]:
model=inceptionv3()
model.compile(loss='categorical_crossentropy',
                           optimizer=AdaBound(lr=1e-3, final_lr=0.1),
                           metrics=['accuracy'])
model.fit(train_face, y_train,epochs=100,batch_size=32,shuffle=True)








Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/100
5764/5764 [==============================] - 48s 8ms/step - loss: 4.2595 - acc: 0.0344
Epoch 2/100
5764/5764 [==============================] - 26s 4ms/step - loss: 4.0888 - acc: 0.0576
Epoch 3/100
5764/5764 [==============================] - 27s 5ms/step - loss: 3.8736 - acc: 0.0796
Epoch 4/100
5764/5764 [==============================] - 27s 5ms/step - loss: 3.6116 - acc: 0.1142
Epoch 5/100
5764/5764 [==============================] - 27s 5ms/step - loss: 3.4089 - acc: 0.1365
Epoch 6/100
5764/5764 [==============================] - 27s 5ms/step - loss: 3.2412 - acc: 0.1556
Epoch 7/100
5764/5764 [==============================] - 27s 5ms/step - loss: 3.1004 - acc: 0.1804
Epoch 8/100
5764/5764 [==============================] - 27s 5ms/step - loss: 2.9859 - acc: 0.1985
Epoch 9/100
5764/5764 [==============================] - 27s 5ms/step - loss: 2.8838 - acc: 0.2148
Epoch 10

**Saving the softmax model**

In [0]:
model.save('facerecognition_softmax.h5')

**Evaluating loss and accuracy on the test set**

In [17]:
score=model.evaluate(test_face,test_labels)

681/681 [==============================] - 2s 4ms/step


**Getting the loss and accuracy from the above in a list**

In [18]:
print(score)

[2.668466484319096, 0.4831130693662534]
